![(book cover)](https://covers.oreillystatic.com/images/0636920167433/cat.gif "(book cover)")
# Programming Quantum Computers by O'Reilly Media -  [book info](http://shop.oreilly.com/product/0636920167433.do)  - [all code samples](https://oreilly-qc.github.io)

## Code samples for Chapter 9
These code samples were written by Mariia Mykhailova.

### Example 9-1: Using a QPU to increment a number in RAM

In [1]:
// Example 9-1: Using a QPU to increment a number in RAM

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;

operation IncrementInteger (a : Int, nBits : Int) : Int {
    use register = Qubit[nBits];
    let registerLE = LittleEndian(register);
    // Write the integer into the quantum register (using little endian)
    ApplyPauliFromBitString(PauliX, true, IntAsBoolArray(a, nBits), register);

    // Perform the increment using library operation
    IncrementByInteger(1, registerLE);

    // Read the result from the register and return it
    return MeasureInteger(registerLE);
}

operation IncrementIntegerDemo () : Unit {
    let nBits = 3;
    mutable a = [4, 3, 5, 1];
    Message($"Initial array: {a}");
    
    let indexUpdate = 2;
    set a w/= indexUpdate <- IncrementInteger(a[indexUpdate], nBits);
    Message($"Array after incrementing element {indexUpdate}: {a}");
}

IncrementInteger, IncrementIntegerDemo

In [2]:
%simulate IncrementIntegerDemo

Initial array: [4,3,5,1]
Array after incrementing element 2: [4,3,6,1]


()

### Example 9-2: Using a QPU to increment a number in QRAM

In [3]:
// Example 9-2: Using a QPU to increment a number in QRAM

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;

operation LoadFromQRAM (data : Int[], indexRegister : Qubit[], dataRegister : Qubit[]) : Unit is Adj+Ctl {
    // Mock QRAM loading by using controlled-not gates, with indexRegister as controls
    for i in 0 .. Length(data) - 1 {
        // Convert the data into a transformation we'll apply on dataRegister to encode it
        let encodeAi = ApplyPauliFromBitString(PauliX, true, IntAsBoolArray(data[i], Length(dataRegister)), _);
        ControlledOnInt(i, encodeAi)(indexRegister, dataRegister);
    }
}

operation LoadAndIncrementDemo () : Unit {
    let data = [4, 3, 5, 1];
    use (indexRegister, dataRegister) = (Qubit[2], Qubit[3]);
    // Prepare index register in superposition
    H(indexRegister[0]);
    X(indexRegister[1]);
    Message("Preparing index register in superposition...");
    DumpMachine();

    // Load the data from QRAM using the index register in superposition as indices
    Message("Loading the data from QRAM...");
    LoadFromQRAM(data, indexRegister, dataRegister);
    DumpMachine();

    // Perform the increment using library operation
    Message("Incrementing the data...");
    IncrementByInteger(1, LittleEndian(dataRegister));
    DumpMachine();

    ResetAll(indexRegister + dataRegister);
}

LoadFromQRAM, LoadAndIncrementDemo

In [4]:
%config dump.truncateSmallAmplitudes=true
%simulate LoadAndIncrementDemo

Preparing index register in superposition...


|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0.7071067811865476 + 0𝑖

Loading the data from QRAM...


|7⟩	0.7071067811865476 + 0𝑖
|22⟩	0.7071067811865476 + 0𝑖

Incrementing the data...


|11⟩	0.7071067811865482 + 6.106674209741569E-18𝑖
|26⟩	0.7071067811865482 + 6.106674209741569E-18𝑖

()

### Example 9-3: Preparing amplitude-encoded vectors

In [5]:
// Example 9-3: Preparing amplitude-encoded vectors

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Preparation;

operation AmplitudeEncodingDemo () : Unit {
    let data = [-1.0, 1.0, 1.0, 5.0, 5.0, 6.0, 6.0, 6.0];
    let N = Length(data);
    
    // Convert array of real data points to an array of ComplexPolar data type
    mutable dataComplexPolar = new ComplexPolar[N];
    for i in 0 .. N - 1 {
        set dataComplexPolar w/= i <- ComplexAsComplexPolar(Complex(data[i], 0.0));
    }
    
    // Allocate register of right size to amplitude encode vector
    let nQubits = Round(Lg(IntAsDouble(N)));
    use register = Qubit[nQubits];
    let phase = PrepareArbitraryStateCP(dataComplexPolar, LittleEndian(register));
    DumpMachine();
    ResetAll(register);
}

AmplitudeEncodingDemo

In [6]:
%simulate AmplitudeEncodingDemo

|0⟩	-0.07881104062391012 + -2.4129220123773967E-17𝑖
|1⟩	0.07881104062391013 + -2.4707094197063485E-18𝑖
|2⟩	0.07881104062391003 + -1.8823165617375972E-18𝑖
|3⟩	0.39405520311955056 + -8.566688063721728E-17𝑖
|4⟩	0.3940552031195506 + -2.1499093428658467E-17𝑖
|5⟩	0.4728662437434608 + 7.371892742682323E-19𝑖
|6⟩	0.47286624374346053 + 1.5420590719168087E-17𝑖
|7⟩	0.47286624374346076 + -1.3194685480583507E-17𝑖

()